In [1]:
import socket
import threading
import time


class TCPClient:
    def __init__(self, host="127.0.0.1", port=9999):
        self.host = host
        self.port = port
        self.client_socket = None
        self.running = False

    def connect(self):
        """连接到服务器"""
        try:
            self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.client_socket.connect((self.host, self.port))
            print(f"成功连接到服务器 {self.host}:{self.port}")
            self.running = True

            # 启动接收消息的线程
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()

            return True
        except Exception as e:
            print(f"连接服务器失败: {e}")
            return False

    def disconnect(self):
        """断开与服务器的连接"""
        self.running = False
        if self.client_socket:
            try:
                self.client_socket.close()
                print("已断开与服务器的连接")
            except Exception as e:
                print(f"断开连接时发生错误: {e}")

    def send_message(self, message):
        """发送消息到服务器"""
        if not self.client_socket:
            print("未连接到服务器")
            return False

        try:
            self.client_socket.send(message.encode("utf-8"))
            return True
        except Exception as e:
            print(f"发送消息失败: {e}")
            return False

    def receive_messages(self):
        """接收服务器消息的循环"""
        while self.running:
            try:
                data = self.client_socket.recv(1024)
                if not data:
                    print("服务器已断开连接")
                    self.disconnect()
                    break

                message = data.decode("utf-8")
                print(f"收到服务器消息: {message}")
            except Exception as e:
                if self.running:
                    print(f"接收消息时发生错误: {e}")
                    self.disconnect()
                break

    def start_interactive(self):
        """启动交互式消息发送循环"""
        if not self.connect():
            return

        try:
            while self.running:
                message = input("请输入要发送的消息（输入 'quit' 退出）: ")
                if message.lower() == "quit":
                    break

                if not self.send_message(message):
                    break

                # 短暂延迟以防止消息发送过快
                time.sleep(0.1)

        finally:
            self.disconnect()

In [3]:
# 使用示例
client = TCPClient("127.0.0.1", 8080)
client.start_interactive()

成功连接到服务器 127.0.0.1:8080
收到服务器消息: 服务器已收到消息: 11111
已断开与服务器的连接
